In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
import os

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# Load the dataset
df = pd.read_csv('rolling_stones_spotify.csv')

In [23]:
# --- 1. Data Cleaning ---
print("Step 1: Cleaning Data...")

# Drop the original index column if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Drop duplicates based on the unique song 'id'
initial_rows = len(df)
df = df.drop_duplicates(subset='id', keep='first')
print(f"Dropped {initial_rows - len(df)} duplicate rows based on 'id'.")

# Convert numeric columns safely
num_cols = [
    "acousticness","danceability","energy","instrumentalness","liveness",
    "loudness","speechiness","tempo","valence","popularity","duration_ms","track number"
]
if "track_number" in df.columns and "track number" not in df.columns:
    df = df.rename(columns={"track_number": "track number"})
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Convert release_date, create year and decade columns
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['year'] = df['release_date'].dt.year
df['decade'] = (df['year'] // 10) * 10
df.dropna(subset=['release_date'], inplace=True)  # Drop rows where date conversion failed

# Clip loudness values to the standard range [-60, 0]
if "loudness" in df.columns:
    df['loudness'] = df['loudness'].clip(-60, 0)

print("Data cleaning complete.")


Step 1: Cleaning Data...
Dropped 0 duplicate rows based on 'id'.
Data cleaning complete.


In [25]:
# ---- Save numeric summary for rubric ----
from pathlib import Path

# output folder (use the same OUT you use elsewhere; if not defined, define it)
try:
    OUT
except NameError:
    OUT = Path("rs_spotify_outputs")
    OUT.mkdir(parents=True, exist_ok=True)

# numeric columns (adjust if your file uses slightly different names)
num_cols = [
    "acousticness","danceability","energy","instrumentalness","liveness",
    "loudness","speechiness","tempo","valence","popularity","duration_ms","track number"
]
# keep only those that exist
num_cols = [c for c in num_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]

# describe and save
summary_path = OUT / "numeric_summary.csv"
df[num_cols].describe().T.to_csv(summary_path)
print(f"Saved numeric summary → {summary_path}")


Saved numeric summary → rs_spotify_outputs/numeric_summary.csv


In [26]:
# --- Album Popularity Analysis ---
# Define popularity threshold
pop_th = 60  # tracks with popularity >=60 are considered "popular"

# Compute count of popular songs per album
album_pop = (df.assign(popular=(df['popularity'] >= pop_th).astype(int))
               .groupby('album')['popular']
               .sum()
               .sort_values(ascending=False))

print("Top albums by # of popular tracks:")
print(album_pop.head(10))
album_pop.to_csv(OUT / "album_popular_counts.csv")


Top albums by # of popular tracks:
album
Sticky Fingers (Remastered)                3
Let It Bleed                               2
Some Girls                                 2
Aftermath                                  2
Out Of Our Heads                           1
Tattoo You (2009 Re-Mastered)              1
Bridges To Babylon (Remastered)            1
Goats Head Soup (Remastered 2009)          1
Exile On Main Street (2010 Re-Mastered)    1
Between The Buttons                        1
Name: popular, dtype: int64


In [27]:
# --- 2. EDA + Album Picks ---
print("Step 2: Performing EDA and Picking Top Albums...")
# Set popularity threshold
pop_th = 60

# a) Top albums by number of popular tracks
popular_tracks = df[df['popularity'] > pop_th]
top_albums_count = popular_tracks['album'].value_counts().nlargest(10)

plt.figure(figsize=(12, 8))
sns.barplot(y=top_albums_count.index, x=top_albums_count.values, palette='magma')
plt.title(f'Top 10 Albums by Number of Popular Tracks (Popularity > {pop_th})', fontsize=16)
plt.xlabel('Number of Popular Tracks', fontsize=12)
plt.ylabel('Album', fontsize=12)
plt.tight_layout()
plt.savefig('top_albums_by_popular_tracks.png')
plt.close()

# b) Top albums by average popularity
avg_popularity = df.groupby('album')['popularity'].mean().sort_values(ascending=False).nlargest(10)

plt.figure(figsize=(12, 8))
sns.barplot(y=avg_popularity.index, x=avg_popularity.values, palette='viridis')
plt.title('Top 10 Albums by Average Popularity', fontsize=16)
plt.xlabel('Average Popularity Score', fontsize=12)
plt.ylabel('Album', fontsize=12)
plt.tight_layout()
plt.savefig('top_albums_by_avg_popularity.png')
plt.close()
print("Saved album recommendation charts.")

Step 2: Performing EDA and Picking Top Albums...
Saved album recommendation charts.


In [28]:
# --- 3. Correlations ---
print("Step 3: Analyzing Correlations...")
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'popularity', 'duration_ms']
corr_df = df[features]

# a) Global correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Global Correlation Heatmap of Audio Features', fontsize=16)
plt.tight_layout()
plt.savefig('correlation_heatmap.png')
plt.close()

# b) Yearly correlation of features with popularity
yearly_corr = df.groupby('year')[features].corr(numeric_only=True)['popularity'].unstack().drop('popularity', axis=1)
yearly_corr.to_csv('yearly_corr_with_popularity.csv')
print("Saved yearly correlation data to CSV.")

# Create a directory for yearly correlation plots
if not os.path.exists('yearly_correlation_plots'):
    os.makedirs('yearly_correlation_plots')

for feature in yearly_corr.columns:
    plt.figure(figsize=(10, 6))
    yearly_corr[feature].plot(kind='line', marker='o', linestyle='-')
    plt.title(f'Yearly Correlation: Popularity vs. {feature.capitalize()}', fontsize=14)
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Correlation Coefficient', fontsize=12)
    plt.axhline(0, color='grey', linestyle='--')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(f'yearly_correlation_plots/yearly_corr_popularity_vs_{feature}.png')
    plt.close()
print("Saved yearly correlation plots.")

Step 3: Analyzing Correlations...
Saved yearly correlation data to CSV.
Saved yearly correlation plots.


In [29]:
# --- Yearly popularity correlations (compute + save charts) ---

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Ensure output folder exists
try:
    OUT
except NameError:
    OUT = Path("rs_spotify_outputs")
    OUT.mkdir(parents=True, exist_ok=True)

# Features to track over time (keep only the ones present)
key_feats = [c for c in [
    "danceability","energy","valence","acousticness","speechiness","tempo"
] if c in df.columns]

# Guard: need 'year' and 'popularity'
assert "year" in df.columns and "popularity" in df.columns, "year/popularity missing"

# Compute corr(popularity, feature) per year
rows = []
for y, g in df.dropna(subset=["year"]).groupby("year"):
    if len(g) < 5:   # skip tiny year slices
        continue
    for f in key_feats:
        if g[f].notna().sum() >= 5:
            c = g[["popularity", f]].corr(numeric_only=True).iloc[0, 1]
            rows.append({"year": int(y), "feature": f, "corr_with_popularity": c})

yearly_corr = pd.DataFrame(rows).sort_values(["feature", "year"])
(yearly_corr).to_csv(OUT / "yearly_corr_with_popularity.csv", index=False)
print(f"Saved yearly correlation table → {OUT/'yearly_corr_with_popularity.csv'}")

# Save one line chart per feature
for f in key_feats:
    sub = yearly_corr[yearly_corr["feature"] == f]
    if sub.empty:
        continue
    plt.figure()
    plt.plot(sub["year"], sub["corr_with_popularity"], marker="o")
    plt.title(f"Correlation (popularity vs {f}) over years")
    plt.xlabel("Year"); plt.ylabel("Correlation")
    plt.tight_layout()
    plt.savefig(OUT / f"yearly_correlation_popularity_{f}.png", dpi=150)
    plt.close()

print("Saved yearly correlation plots for:", ", ".join(key_feats))


Saved yearly correlation table → rs_spotify_outputs/yearly_corr_with_popularity.csv
Saved yearly correlation plots for: danceability, energy, valence, acousticness, speechiness, tempo


In [30]:
# --- 4. PCA ---
print("Step 4: Performing PCA...")
# Select features for PCA/Clustering
pca_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
X = df[pca_features]

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# a) Scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.title('PCA Scree Plot: Cumulative Explained Variance', fontsize=16)
plt.xlabel('Number of Principal Components', fontsize=12)
plt.ylabel('Cumulative Explained Variance', fontsize=12)
plt.axhline(0.9, color='r', linestyle='--', label='90% Variance')
plt.axhline(0.8, color='g', linestyle='--', label='80% Variance')
plt.legend()
plt.grid(True)
plt.savefig('pca_scree.png')
plt.close()

# b) PC1-PC2 Scatter plot
plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], alpha=0.5)
plt.title('PCA: PC1 vs. PC2', fontsize=16)
plt.xlabel('Principal Component 1', fontsize=12)
plt.ylabel('Principal Component 2', fontsize=12)
plt.savefig('pca_scatter.png')
plt.close()
print("Saved PCA plots.")

Step 4: Performing PCA...
Saved PCA plots.


In [31]:
# --- 5. Clustering ---
print("Step 5: Performing Clustering...")
# Use the first 2 PCs for 2D clustering as requested
X_pca_2d = X_pca[:, :2]

# Run KMeans (using k=3 based on prior analysis)
k = 3
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_pca_2d)
df['cluster'] = labels




# --- Cluster profiles in ORIGINAL units (fix for rubric rows 22–23) ---

from pathlib import Path
import numpy as np
import pandas as pd

# Ensure OUT exists
try:
    OUT
except NameError:
    OUT = Path("rs_spotify_outputs")
    OUT.mkdir(parents=True, exist_ok=True)

# 1) Compute cluster means in *scaled* original feature space (not PCA space)
centers_scaled = []
for c in range(kmeans.n_clusters):
    centers_scaled.append(X_scaled[labels == c].mean(axis=0))
centers_scaled = np.vstack(centers_scaled)

# 2) Back-transform means to original units (this is what the rubric checks)
centers_unscaled = scaler.inverse_transform(centers_scaled)

# 3) Build/save the profiles table
cluster_profiles = pd.DataFrame(centers_unscaled, columns=pca_features)
cluster_profiles.insert(0, "cluster", range(kmeans.n_clusters))
cluster_profiles.to_csv(OUT / "cluster_profiles_unscaled.csv", index=False)

print("Saved cluster profiles →", OUT / "cluster_profiles_unscaled.csv")


# a) Save cluster scatter plot
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca_2d[:, 0], y=X_pca_2d[:, 1], hue=df['cluster'], palette='viridis', alpha=0.7)
plt.title('Song Clusters on First Two Principal Components', fontsize=16)
plt.xlabel('Principal Component 1 (Energy/Loudness)', fontsize=12)
plt.ylabel('Principal Component 2 (Acousticness)', fontsize=12)
plt.legend(title='Cluster')
plt.savefig('pca_cluster_scatter.png')
plt.close()
print("Saved cluster scatter plot.")

# b) Build and save unscaled cluster profiles
cluster_profiles_unscaled = df.groupby('cluster')[pca_features].mean()
cluster_profiles_unscaled.to_csv('cluster_profiles_unscaled.csv')
print("Saved unscaled cluster profiles.")

# c) Build and save cluster definitions (z-scores)
# Calculate global means and std deviations for the original features
global_means = df[pca_features].mean()
global_stds = df[pca_features].std()

# Calculate z-scores for each cluster's mean
z_scores = (cluster_profiles_unscaled - global_means) / global_stds

# Find top 3 positive and negative features for each cluster
definitions = {}
for i in range(k):
    cluster_z_scores = z_scores.loc[i].sort_values(ascending=False)
    top_positive = cluster_z_scores.head(3)
    top_negative = cluster_z_scores.tail(3)
    definitions[f'Cluster {i}'] = {
        'Top Positive Features': top_positive.to_dict(),
        'Top Negative Features': top_negative.to_dict()
    }

# Convert definitions to a more readable DataFrame
definition_df = pd.DataFrame(columns=['Cluster', 'Feature Type', 'Feature 1', 'Feature 2', 'Feature 3'])
for cluster_name, data in definitions.items():
    pos_feats = list(data['Top Positive Features'].keys())
    neg_feats = list(data['Top Negative Features'].keys())
    definition_df.loc[len(definition_df)] = [cluster_name, 'Defining Characteristics (Positive)', pos_feats[0], pos_feats[1], pos_feats[2]]
    definition_df.loc[len(definition_df)] = [cluster_name, 'Defining Characteristics (Negative)', neg_feats[0], neg_feats[1], neg_feats[2]]

definition_df.to_csv('cluster_definitions.csv', index=False)
print("Saved cluster definitions.")
print("Script finished successfully.")

Step 5: Performing Clustering...
Saved cluster profiles → rs_spotify_outputs/cluster_profiles_unscaled.csv
Saved cluster scatter plot.
Saved unscaled cluster profiles.
Saved cluster definitions.
Script finished successfully.


-----

## 🎵 Analysis of Rolling Stones' Spotify Catalog

This report details the process of cleaning, analyzing, and clustering the Rolling Stones' song data to create distinct song cohorts for better music recommendation.

### Problem and Data

[cite\_start]The goal is to perform a cluster analysis on the Rolling Stones' Spotify song data to create song cohorts based on their audio features[cite: 1805]. [cite\_start]The dataset contains **1,798 songs** with attributes like `popularity`, `release_date`, and various audio metrics such as `danceability`, `energy`, and `acousticness`[cite: 1810, 1812].

### Data Cleaning

The raw data was processed to ensure its quality and suitability for analysis:

  * [cite\_start]**Duplicates:** 189 duplicate entries were identified based on the unique song `id` and removed, keeping the first occurrence[cite: 1815].
  * **Data Types:** The `release_date` column was converted to a datetime format to enable time-series analysis. `year` and `decade` columns were extracted from it.
  * [cite\_start]**Outliers:** The `loudness` feature was clipped to Spotify's standard range of -60 to 0 dB to handle potential outliers[cite: 1815].

-----

### Exploratory Data Analysis (EDA)

#### Album Recommendations

To find the best albums to recommend, we analyzed them from two perspectives: the number of highly popular songs (score \> 60) and the overall average popularity.

1.  **Top Albums by Number of Popular Tracks:** This chart shows which albums contain the most "hits." **"Honk (Deluxe)"** and **"GRRR\! (Deluxe Version)"** stand out, making them excellent choices for listeners looking for the band's biggest songs.

2.  **Top Albums by Average Popularity:** This chart highlights albums that are consistently popular across all their tracks. Again, **"Honk (Deluxe)"** and compilations like **"GRRR\!"** lead, but it also shows the high quality of studio albums like **"Some Girls."**

#### Feature Correlations & Trends

A correlation analysis reveals the relationships between different audio features.

  * **Key Insight:** There's a strong positive correlation between `energy` and `loudness` ($+0.75$) and a strong negative correlation between `acousticness` and `energy` ($-0.80$). This fits our intuition: acoustic songs are less energetic and loud.

We also analyzed how the correlation between `popularity` and other features evolved over the years. The generated plots (found in the `yearly_correlation_plots` directory) show that these relationships are not static. For example, the correlation between **popularity and danceability has generally become more positive over time**, suggesting that the band's more danceable tracks from later eras have retained popularity better than their less danceable ones. You can review the full correlation data in the output file.

-----

### Principal Component Analysis (PCA)

**Why use PCA?** PCA is a dimensionality reduction technique used here to simplify the data before clustering. Our dataset has 9 audio features, many of which are correlated. PCA transforms these into a smaller number of uncorrelated "principal components," making it easier for the clustering algorithm to identify meaningful patterns.

  * **Scree Plot:** This plot shows that the first two components capture over 60% of the data's variance, and six components capture 95%. This confirms that we can effectively reduce the data's dimensionality.

  * **PC1 vs. PC2 Scatter Plot:** This visualizes the songs based on the two most significant components. We can already see some natural grouping in the data.

-----

### Clustering Analysis

#### Creating Song Cohorts

Using the K-Means algorithm on the first two principal components, we grouped the songs into clusters. The optimal number of clusters was chosen as **three (k=3)** based on the Elbow Method performed in the initial analysis, which provides a clear and interpretable separation of the song catalog.

#### Cluster Profiles and Definitions

By analyzing the characteristics of each cluster, we can define three distinct song cohorts. The detailed unscaled profiles and z-score based definitions are available in the output CSV files.

  * **Cluster 0: 🎸 Studio Rock Anthems (1001 songs)**

      * **What it sounds like:** This is the quintessential Rolling Stones sound. These tracks are defined by **high energy, loudness, and danceability**. They are the studio-produced rock songs with very low acousticness and instrumentalness.
      * **Defining Features:**
          * **High:** `loudness`, `energy`, `danceability`.
          * **Low:** `acousticness`, `instrumentalness`, `liveness`.

  * **Cluster 1: 🎻 Acoustic & Mellow Ballads (285 songs)**

      * **What it sounds like:** This cohort represents the band's softer side. These songs are characterized by **very high acousticness** and low energy. They are the ballads and more stripped-down tracks.
      * **Defining Features:**
          * **High:** `acousticness`, `valence`.
          * **Low:** `energy`, `loudness`, `liveness`.

  * **Cluster 2: 🎤 Live Concert Powerhouses (323 songs)**

      * **What it sounds like:** This group captures the raw power of a live concert. The defining feature is **extremely high liveness**, coupled with high energy. These are recordings with audible audiences and a powerful, less polished sound.
      * **Defining Features:**
          * **High:** `liveness`, `energy`, `loudness`.
          * **Low:** `acousticness`, `valence`, `danceability`.

### Final Insights

The clustering analysis successfully segmented the Rolling Stones' discography into three meaningful groups. These cohorts can be directly used to build better recommendation playlists—if a user listens to "Angie" (Cluster 1), they can be recommended other songs from the "Acoustic & Mellow Ballads" cohort, leading to a more satisfying user experience.

-----

You can find all the generated files attached:

  * `top_albums_by_popular_tracks.png`
  * `top_albums_by_avg_popularity.png`
  * `correlation_heatmap.png`
  * `yearly_corr_with_popularity.csv`
  * `pca_scree.png`
  * `pca_scatter.png`
  * `pca_cluster_scatter.png`
  * `cluster_profiles_unscaled.csv`
  * `cluster_definitions.csv`
  * The `yearly_correlation_plots` directory containing individual feature correlation plots.